In [11]:
import pandas as pd
import numpy as np
import matplotlib as plt
from sklearn.model_selection import train_test_split

from keras.utils.np_utils import to_categorical
from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten, Conv2D, MaxPool2D, BatchNormalization
from keras.preprocessing.image import ImageDataGenerator
from keras.callbacks import LearningRateScheduler
from keras.datasets import mnist

In [2]:
(x_train, y_train), (x_test, y_test) = mnist.load_data()

11493376/11490434 [==============================] - 0s 0us/step


In [3]:
# data prep
X_train = x_train / 255.0
X_test = x_test / 255.0
X_train = X_train.reshape(-1,28,28,1)
X_test = X_test.reshape(-1,28,28,1)
Y_train = to_categorical(y_train, num_classes = 10)

In [4]:
# enlargement of training set through data augmentation
datagen = ImageDataGenerator(
        rotation_range=15,
        zoom_range = 0.15,  
        width_shift_range=0.1, 
        height_shift_range=0.1)

In [5]:
# setting up the cnn ensamble
nets = 15
model = [0] *nets
for j in range(nets):
    model[j] = Sequential()

    model[j].add(Conv2D(32, kernel_size = 3, activation='relu', input_shape = (28, 28, 1)))
    model[j].add(BatchNormalization())
    model[j].add(Conv2D(32, kernel_size = 3, activation='relu'))
    model[j].add(BatchNormalization())
    model[j].add(Conv2D(32, kernel_size = 5, strides=2, padding='same', activation='relu'))
    model[j].add(BatchNormalization())
    model[j].add(Dropout(0.4))

    model[j].add(Conv2D(64, kernel_size = 3, activation='relu'))
    model[j].add(BatchNormalization())
    model[j].add(Conv2D(64, kernel_size = 3, activation='relu'))
    model[j].add(BatchNormalization())
    model[j].add(Conv2D(64, kernel_size = 5, strides=2, padding='same', activation='relu'))
    model[j].add(BatchNormalization())
    model[j].add(Dropout(0.4))

    model[j].add(Conv2D(128, kernel_size = 4, activation='relu'))
    model[j].add(BatchNormalization())
    model[j].add(Flatten())
    model[j].add(Dropout(0.4))
    model[j].add(Dense(10, activation='softmax'))

    # compiling model
    model[j].compile(optimizer="adam", loss="categorical_crossentropy", metrics=["accuracy"])

In [7]:
# decrease learning rateby by 0.95 each epoch
annealer = LearningRateScheduler(lambda x: 1e-3 * 0.95 ** x)

# train cnns and display accuracies
epochs = 30
history = [0] * nets
results = [0] * nets
for j in range(nets):
    X_train2, X_val2, Y_train2, Y_val2 = train_test_split(X_train, Y_train, test_size = 0.1)
    history[j] = model[j].fit_generator(datagen.flow(X_train2,Y_train2, batch_size=64),
      epochs = epochs, steps_per_epoch = X_train2.shape[0]//64,
      validation_data = (X_val2,Y_val2), callbacks=[annealer], verbose=0)
    print("CNN {0:d}: Epochs={1:d}, Train accuracy={2:.5f}, Validation accuracy={3:.5f}".format
      (j+1,epochs,history[j].history['accuracy'][epochs-1],history[j].history['val_accuracy'][epochs-1]))
    
    # predicting digits for cnn j based on 10k mnist test set
    results[j] = model[j].predict(X_test)
    results2 = np.argmax(results[j],axis = 1)

    # calculating cnn j accuracy
    c=0
    for i in range(10000):
        if results2[i]!=y_test[i]:
            c +=1
    print("CNN %d: Test accuracy = %f" % (j+1,1-c/10000.))

CNN 1: Epochs=30, Train accuracy=0.99509, Validation accuracy=0.99750
CNN 1: Test accuracy = 0.996800
CNN 2: Epochs=30, Train accuracy=0.99329, Validation accuracy=0.99717
CNN 2: Test accuracy = 0.996400
CNN 3: Epochs=30, Train accuracy=0.99320, Validation accuracy=0.99717
CNN 3: Test accuracy = 0.996600
CNN 4: Epochs=30, Train accuracy=0.99333, Validation accuracy=0.99667
CNN 4: Test accuracy = 0.996600
CNN 5: Epochs=30, Train accuracy=0.99397, Validation accuracy=0.99500
CNN 5: Test accuracy = 0.995900
CNN 6: Epochs=30, Train accuracy=0.99394, Validation accuracy=0.99750
CNN 6: Test accuracy = 0.996700
CNN 7: Epochs=30, Train accuracy=0.99355, Validation accuracy=0.99533
CNN 7: Test accuracy = 0.996500
CNN 8: Epochs=30, Train accuracy=0.99366, Validation accuracy=0.99517
CNN 8: Test accuracy = 0.996600
CNN 9: Epochs=30, Train accuracy=0.99362, Validation accuracy=0.99717
CNN 9: Test accuracy = 0.996700
CNN 10: Epochs=30, Train accuracy=0.99388, Validation accuracy=0.99667
CNN 10: Tes

In [8]:
# predicting digits for ensemble based on 10k mnist test set
results2 = np.zeros( (X_test.shape[0],10) )
for j in range(nets):
    results2 = results2 + results[j]
results2 = np.argmax(results2,axis = 1)
 
# calculating ensemble accuracy
c=0
for i in range(10000):
    if results2[i]!=y_test[i]:
        c +=1
print("Ensemble Accuracy = %f" % (1-c/10000.))

Ensemble Accuracy = 0.997600
